In [42]:
# import libraries
import pandas as pd
import pyarrow.parquet as pq
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import dask.dataframe as dd
#import geopandas as gpd
# import osmnx as ox
import json
from datetime import datetime, timedelta, date
import os
import time
import sys
import requests
# import sidetable
import warnings
warnings.filterwarnings('ignore')

In [43]:
sys.tracebacklimit = 0 # turn off long tracebacks on error messages
#sys.tracebacklimit = None # use to turn tracebacks back on if needed

In [44]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [45]:
#extra columns

#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_casirivimab_plus_imdevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_bamlanivimab_plus_etesevimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_sotrovimab_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_evusheld_number_of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_molnupiravir_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_this_facility',
#                 'therapeutic_paxlovid_number_ of_residents_treated_from_stock_stored_at_another_facility',
#                 'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_this_facility',
#                'therapeutic_bebtelovimab_number_of_residents_treated from_stock_stored_at_another_facility',

In [46]:
pfile = pq.read_table("data/data_pre_proc/nh_all_Jun27_21_Jan1_23_concat.parquet.gzip")
print("Column names: {}".format(pfile.column_names))
print("Schema: {}".format(pfile.schema))

Column names: ['week_ending', 'federal_provider_number', 'provider_name', 'provider_address', 'provider_city', 'provider_state', 'provider_zip_code', 'provider_phone_number', 'county', 'submitted_data', 'passed_quality_assurance_check', 'residents_weekly_admissions_covid_19', 'residents_total_admissions_covid_19', 'residents_weekly_confirmed_covid_19', 'residents_total_confirmed_covid_19', 'residents_weekly_all_deaths', 'residents_total_all_deaths', 'residents_weekly_covid_19_deaths', 'residents_total_covid_19_deaths', 'number_of_all_beds', 'total_number_of_occupied_beds', 'staff_weekly_confirmed_covid_19', 'staff_total_confirmed_covid_19', 'staff_weekly_covid_19_deaths', 'staff_total_covid_19_deaths', 'Staff_Shortage', 'shortage_of_nursing_staff', 'shortage_of_clinical_staff', 'shortage_of_aides', 'shortage_of_other_staff', 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents', 'weekly_resident_covid_19_deaths_per_1_000_residents', 'total_resident_confirmed_covid_19_cases_per

In [47]:
# set keep only needed cols
needed_cols = ['week_ending',
                'federal_provider_number',
                'provider_name',
                'provider_address',
                'provider_city',
                'provider_state',
                'provider_zip_code',
                'provider_phone_number',
                'county',
                'submitted_data',
                'passed_quality_assurance_check',
                'residents_weekly_confirmed_covid_19',
                'residents_weekly_all_deaths',
                'residents_weekly_covid_19_deaths',
                'number_of_all_beds',
                'total_number_of_occupied_beds',
                'staff_weekly_confirmed_covid_19',
                'staff_weekly_covid_19_deaths',
                'shortage_of_nursing_staff',
                'shortage_of_clinical_staff',
                'shortage_of_aides',
                'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
                'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
                'number_of_residents_with_new_influenza',
                'number_of_staff_and_or_personnel_with_new_influenza',
                'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time',
                'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time',
                'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time',
                'number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
                'recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines',
                'number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time',
                'recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time',
                'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines',
                'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines',
                'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines']



In [48]:
nh_latest = pfile.to_pandas()

In [49]:
nh_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214594 entries, 0 to 1214593
Columns: 121 entries, week_ending to Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
dtypes: object(121)
memory usage: 1.1+ GB


In [50]:
nh_latest_sub = nh_latest[needed_cols]

In [51]:
nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,0,,,,0,0,0,0,N,53,49,4,,92.453,92.453,81,70,10,86.42,87.5,,48,,97.959,47,,88.679,24,,34.286,24,,29.63,34.286
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,

In [52]:
nh_latest_sub.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
1214589,2022-11-27,745003,NEURORESTORATIVE TEXAS,2301 W. CAMPBELL RD.,GARLAND,TX,75044,5124382630,Dallas,Y,N,0,0,0,12,2,0,0,,,,0,0,0,0,Y,2,0,2,,0,0,12,10,2,83.333,83.333,,0,,0,0,,0,0,,0,1,,8.333,10
1214590,2022-11-27,745004,THE BRIXTON AT HORSESHOE BAY,15101 WEST FM 2147,HORSESHOE BAY,TX,78657,7135531321,Burnet,Y,Y,0,0,0,120,48,0,0,,,,0,0,0,0,Y,48,45,3,,93.

In [53]:
list(nh_latest_sub.columns)

['week_ending',
 'federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'submitted_data',
 'passed_quality_assurance_check',
 'residents_weekly_confirmed_covid_19',
 'residents_weekly_all_deaths',
 'residents_weekly_covid_19_deaths',
 'number_of_all_beds',
 'total_number_of_occupied_beds',
 'staff_weekly_confirmed_covid_19',
 'staff_weekly_covid_19_deaths',
 'shortage_of_nursing_staff',
 'shortage_of_clinical_staff',
 'shortage_of_aides',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
 'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result',
 'number_of_residents_with_new_influenza',
 'number_of_staff_and_or_personnel_with_new_influenza',
 'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel',
 'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week',
 'number_of_residents_staying_in_this_facility_

In [54]:
nh_latest_sub.sort_values(by=['week_ending']).head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
87317,2021-06-27,415091,STEERE HOUSE NURSING AND REHABILITATION CTR,100 BORDEN STREET,PROVIDENCE,RI,02903,4014547970,Providence,Y,Y,0,0,0,120,95,0,0,N,N,N,0,0,0,0,,100,95,,,95,95,163,134,,82.209,82.209,,,,0,,,0,,,0,,,0,0
85706,2021-06-27,365937,OHMAN FAMILY LIVING AT BRIAR,15950 PIERCE ST,MIDDLEFIELD,OH,44062,4406325241,Geauga,Y,Y,0,0,0,96,81,0,0,N,N,N,0,0,

In [55]:
nh_latest_sub.sort_values(by=['week_ending']).tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines
20150,2023-01-01,185297,SANSBURY CARE CENTER,2625 BARDSTOWN ROAD,SAINT CATHARINE,KY,40061,8593363974,Washington,Y,Y,0,0,0,59,39,1,0,,,,0,1,0,0,Y,39,39,0,100,100,100,,,,,,98.551,39,100,100,32,82.051,82.051,,52.941,,,19.565,,
20151,2023-01-01,185298,HICKS GOLDEN YEARS NURSING HOME,1901 WEST HIGHWAY 90 BYPASS,MONTICELLO,KY,42633,6063486034,Wayne,Y,Y,0,0,0,59,5

In [56]:
#check for dup rows
nh_latest_sub.loc[nh_latest_sub.duplicated(),]

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,staff_weekly_confirmed_covid_19,staff_weekly_covid_19_deaths,shortage_of_nursing_staff,shortage_of_clinical_staff,shortage_of_aides,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result,number_of_residents_with_new_influenza,number_of_staff_and_or_personnel_with_new_influenza,Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines


In [57]:
# rename long cols
nh_latest_sub = nh_latest_sub.rename({
    'provider_zip_code':'provider_zip',
    'passed_quality_assurance_check':'data_passed_qa',
    'residents_weekly_confirmed_covid_19':'res_wk_confirmed_c19',
    'residents_weekly_all_deaths': 'res_wk_all_deaths',
    'residents_weekly_covid_19_deaths':'res_wk_c19_deaths',
    'number_of_all_beds':'num_all_beds',
    'total_number_of_occupied_beds':'num_occ_beds',
    'staff_weekly_confirmed_covid_19':'staff_wk_conf_c19',
     'staff_weekly_covid_19_deaths':'staff_wk_c19_deaths',
    'shortage_of_nursing_staff':'short_nrsg_staff',
    'shortage_of_aides':'short_aides',
    'weekly_resident_confirmed_covid_19_cases_per_1_000_residents':'wk_res_confirmed_c19_per_1K',
    'number_of_staff_and_or_personnel_with_a_new_positive_covid_19_test_result':'num_staff_new_pos_c19_test',
    'number_of_residents_with_new_influenza':'num_res_new_flu',
    'number_of_staff_and_or_personnel_with_new_influenza':'num_staff_pers_new_flu',
    'Able_to_Test_or_Obtain_Resources_to_Test_All_Residents_Staff_and_or_Personnel':'can_test_all_res_staff_pers',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week':'num_res_in_facility',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_res_comp_c19_vax_anytime',
    'number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_res_decln_c19_vax_anytime',
    'recent_percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'rec_pct_res_comp_c19_vax_anytime',
    'percentage_of_current_residents_who_received_a_completed_covid_19_vaccination_at_any_time':'pct_res_comp_c19_vax_anytime',
    'Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_res_utd_c19_vax',
    'Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_res_utd_c19_vax',
    'Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines':'pct_res_utd_c19_vax',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week':'num_hcp',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time':'num_hcp_comp_c19_vax_anytime',
    'number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_were_offered_but_declined_a_covid_19_vaccine_at_any_time':'num_hcp_decln_c19_vax_anytime',
    'recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'rec_pct_hcp_comp_c19_vax_anytime',
    'percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time': 'pct_hcp_comp_c19_vax_anytime',
    'Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines':'num_hcp_utd_c19_vax',
    'Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'rec_pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_utd_c19_vax',
    'Percentage_of_Current_Healthcare_Personnel_with_a_Completed_Vaccination_Up_to_Date_with_COVID_19_Vaccines':'pct_hcp_comp_c19_vax_and_utd',
   }, axis=1)

In [58]:
nh_latest_sub.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,shortage_of_clinical_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,percentage_of_current_residents_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,percentage_of_current_healthcare_personnel_with_no_medical_contraindications_who_received_a_completed_covid_19_vaccination_at_any_time,rec_pct_hcp_comp_c19_vax_anytime,number_of_residents_with_a_completed_vaccination_staying_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_residents_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,pct_res_utd_c19_vax,number_of_healthcare_personnel_with_a_completed_vaccination_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_covid_19_vaccine_booster_at_any_time,recent_percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,percentage_of_current_healthcare_personnel_with_a_completed_vaccination_who_received_a_covid_19_vaccine_booster_at_any_time,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,0,,,,0,0,0,0,N,53,49,4,,92.453,92.453,81,70,10,86.42,87.5,,48,,97.959,47,,88.679,24,,34.286,24,,29.63,34.286
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,81,1,0,,,,0,1,0,0,Y,82,75,6,,91.463,91.463,185,149,31,80.541,81.421,,70,,93.333,59,,71.951,22,,14.765,1,,0.541,0.671
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,35,0,0,,,,0,0,0,0,Y,38,38,0,,100,100,86,81,5,94.186,94.186,,38,,100,35,,92.105,56,,69.136,15,,17.442,18.519
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,1,0,90,75,0,0,,,,0,0,0,0,Y,75,73,2,,97.333,97.333,97,93,2,95.876,95.876,,72,,98.63,72,,96,82,,88.172,30,,30.928,32.258
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,86,0,0,,,,0,0,0,0,Y,92,85,4,,92.391,92.391,139,115,18,82.734,82.734,,75,,88.235,65,,70.652,72,,62.609,5,,3.597,4.348


### What fields do we actualy care about?  From CMS site: https://data.cms.gov/covid-19/covid-19-nursing-home-data:

* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time
* Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines

### Rounding back to the data dictionary for clarity on how these are calculated:
https://data.cms.gov/sites/default/files/2022-11/COVID-19%20Nursing%20Home%20Data%20Dictionary%2011.20.2022.pdf


#### Resident Metrics:
* Recent Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Residents who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b>(Number of Residents Staying in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b> Number of Residents Staying in this Facility for At Least 1 Day This Week) * 100 </b>
<br>
<br>
* <mark> Recent Percentage of Current Residents Up to Date with COVID-19 Vaccines = The value of "Percentage of Residents who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week.
Calculated as follows: <b>(Number of Residents Staying in this Facility
for At Least 1 Day This Week Up to Date with COVID-19 Vaccines</b> /
<b>Number of Residents Staying in this Facility for At Least 1 Day This
Week) * 100</b></mark>
<br>

#### Health Care Personnel Metrics:
* Recent Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time = The value of "Percentage of Current Healthcare Personnel who Received a Completed COVID-19 Vaccination at Any Time" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1 Day This Week who Received a Completed COVID-19 Vaccination at Any Time</b> / <b>Number of All Healthcare Personnel Eligible to Work in this Facility for At Least 1
Day This Week) * 100</b>
<br>
* <mark> Recent Percentage of Current Healthcare Personnel Up to Date with COVID-19 Vaccines = The value of "Percentage of Current Healthcare Personnel who are Up to Date with COVID-19 Vaccines" for the current week, or if blank, for the prior week. Calculated as follows: <b> (Number of Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week Up to Date with
COVID-19 Vaccines </b> / <b> Number of All Healthcare Personnel Eligible to
Work in this Facility for At Least 1 Day This Week) * 100 </b></mark>



### What I care about are the up-to-date metrics. This opens up a new quesiton: What does it mean to be up-to-date on COVID-19 vaccinations now? To have received all vax does plus all available boosters?

### -->answer according to cdc, up-to-date means :Recipient of bivalent booster dose or completion f primary series in the past two months"  https://www.cdc.gov/nhsn/covid19/ltc-vaccination-dashboard.html. "The bivalent booster was included in the up to date definition when it became available to people 12 years of age and older on September 2, 2022."


In [59]:
# subset cols
nh_latest_sub_2 = nh_latest_sub[[
                 'week_ending',
                 'federal_provider_number',
                 'provider_name',
                 'provider_address',
                 'provider_city',
                 'provider_state',
                 'provider_zip',
                 'provider_phone_number',
                 'county',
                 'submitted_data',
                 'data_passed_qa',
                 'res_wk_confirmed_c19',
                 'res_wk_all_deaths',
                 'res_wk_c19_deaths',
                 'num_all_beds',
                 'num_occ_beds',
                 'staff_wk_conf_c19',
                 'staff_wk_c19_deaths',
                 'short_nrsg_staff',
                 'short_aides',
                 'wk_res_confirmed_c19_per_1K',
                 'num_staff_new_pos_c19_test',
                 'num_res_new_flu',
                 'num_staff_pers_new_flu',
                 'can_test_all_res_staff_pers',
                 'num_res_in_facility',
                 'num_res_comp_c19_vax_anytime',
                 'num_res_decln_c19_vax_anytime',
                 'rec_pct_res_comp_c19_vax_anytime',
                 'pct_res_comp_c19_vax_anytime',
                 'num_res_utd_c19_vax',
                 'rec_pct_res_utd_c19_vax',
                 'num_hcp',
                 'num_hcp_comp_c19_vax_anytime',
                 'num_hcp_decln_c19_vax_anytime',
                 'rec_pct_hcp_comp_c19_vax_anytime',
                 'pct_hcp_comp_c19_vax_anytime',
                 'num_hcp_utd_c19_vax',
                 'rec_pct_hcp_utd_c19_vax',
                 'pct_hcp_utd_c19_vax',
                 'pct_hcp_comp_c19_vax_and_utd'
]]

nh_latest_sub_2.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,0,,,0,0,0,0,N,53,49,4,,92.453,47,,81,70,10,,86.42,24,,29.63,34.286
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,81,1,0,,,0,1,0,0,Y,82,75,6,,91.463,59,,185,149,31,,80.541,1,,0.541,0.671
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,35,0,0,,,0,0,0,0,Y,38,38,0,,100,35,,86,81,5,,94.186,15,,17.442,18.519
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,1,0,90,75,0,0,,,0,0,0,0,Y,75,73,2,,97.333,72,,97,93,2,,95.876,30,,30.928,32.258
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,86,0,0,,,0,0,0,0,Y,92,85,4,,92.391,65,,139,115,18,,82.734,5,,3.597,4.348


In [60]:
nh_latest_sub_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1214594 entries, 0 to 1214593
Data columns (total 41 columns):
 #   Column                            Non-Null Count    Dtype 
---  ------                            --------------    ----- 
 0   week_ending                       1214594 non-null  object
 1   federal_provider_number           1214594 non-null  object
 2   provider_name                     1214594 non-null  object
 3   provider_address                  1214594 non-null  object
 4   provider_city                     1214594 non-null  object
 5   provider_state                    1214594 non-null  object
 6   provider_zip                      1214594 non-null  object
 7   provider_phone_number             1214594 non-null  object
 8   county                            1214594 non-null  object
 9   submitted_data                    1214594 non-null  object
 10  data_passed_qa                    1214594 non-null  object
 11  res_wk_confirmed_c19              1214594 non-null

In [61]:
#replace spaces with null
nh_latest_sub_2=nh_latest_sub_2.replace(r'^\s*$', np.nan, regex=True)
nh_latest_sub_2.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,0,0,57,54,0,0,NaN,NaN,0,0,0,0,N,53,49,4,NaN,92.453,47,NaN,81,70,10,NaN,86.42,24,NaN,29.63,34.286
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,0,0,85,81,1,0,NaN,NaN,0,1,0,0,Y,82,75,6,NaN,91.463,59,NaN,185,149,31,NaN,80.541,1,NaN,0.541,0.671
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,0,0,50,35,0,0,NaN,NaN,0,0,0,0,Y,38,38,0,NaN,100,35,NaN,86,81,5,NaN,94.186,15,NaN,17.442,18.519
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,1,0,90,75,0,0,NaN,NaN,0,0,0,0,Y,75,73,2,NaN,97.333,72,NaN,97,93,2,NaN,95.876,30,NaN,30.928,32.258
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,2,0,103,86,0,0,NaN,NaN,0,0,0,0,Y,92,85,4,NaN,92.391,65,NaN,139,115,18,NaN,82.734,5,NaN,3.597,4.348
5,2022-12-25,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0,0,0,149,127,0,0,NaN,NaN,0,0,0,0,Y,119,96,12,NaN,80.672,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-12-25,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0,0,0,124,99,0,0,N,N,0,0,0,0,Y,102,76,17,NaN,74.51,58,NaN,115,107,1,NaN,93.043,20,NaN,17.391,18.692
7,2022-12-25,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,0,0,0,201,82,3,0,Y,Y,0,3,0,0,Y,82,82,0,NaN,100,54,NaN,121,103,17,NaN,85.124,3,NaN,2.479,2.913
8,2022-12-25,015024,"LIMESTONE NURSING AND REHABILITATION CENTER, LLC",1600 WEST HOBBS STREET,ATHENS,AL,35611,2562323461,Limestone,Y,Y,0,1,0,170,119,0,0,NaN,NaN,0,0,0,0,Y,125,101,22,NaN,80.8,44,NaN,186,162,21,NaN,87.097,14,NaN,7.527,8.642
9,2022-12-25,015027,"WETUMPKA HEALTH AND REHABILITATION, LLC",1825 HOLTVILLE ROAD,WETUMPKA,AL,36092,3345675131,Elmore,Y,Y,0,0,0,121,120,0,0,NaN,NaN,0,0,0,0,Y,169,97,25,NaN,57.396,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
#revise nums to float cols
float_cols = [
    'res_wk_confirmed_c19',
    'res_wk_all_deaths',
     'res_wk_c19_deaths',
     'num_all_beds',
     'num_occ_beds',
     'staff_wk_conf_c19',
     'staff_wk_c19_deaths',
     'wk_res_confirmed_c19_per_1K',
     'num_staff_new_pos_c19_test',
     'num_res_new_flu',
     'num_staff_pers_new_flu',
     'num_res_in_facility',
     'num_res_comp_c19_vax_anytime',
     'num_res_decln_c19_vax_anytime',
     'rec_pct_res_comp_c19_vax_anytime',
     'pct_res_comp_c19_vax_anytime',
     'num_res_utd_c19_vax',
     'rec_pct_res_utd_c19_vax',
     'num_hcp',
     'num_hcp_comp_c19_vax_anytime',
     'num_hcp_decln_c19_vax_anytime',
     'rec_pct_hcp_comp_c19_vax_anytime',
     'pct_hcp_comp_c19_vax_anytime',
     'num_hcp_utd_c19_vax',
     'rec_pct_hcp_utd_c19_vax',
     'pct_hcp_utd_c19_vax',
     'pct_hcp_comp_c19_vax_and_utd'
]

nh_latest_sub_2[float_cols]=nh_latest_sub_2[float_cols].astype('float')


In [63]:
nh_latest_sub_2['week_ending'] = pd.to_datetime(nh_latest_sub_2['week_ending'], format='%Y-%m-%d')

# subset to week ending greater than 7-2022
nh_latest_sub_end_22 = nh_latest_sub_2.loc[nh_latest_sub_2['week_ending']>='2022-07-10']
nh_latest_sub_end_22.info()

# create week start date col (should be a Mon)
nh_latest_sub_end_22['week_start'] = nh_latest_sub_end_22['week_ending'] - timedelta(days=6)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 393318 entries, 0 to 1214593
Data columns (total 41 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   week_ending                       393318 non-null  datetime64[ns]
 1   federal_provider_number           393318 non-null  object        
 2   provider_name                     393318 non-null  object        
 3   provider_address                  393318 non-null  object        
 4   provider_city                     393318 non-null  object        
 5   provider_state                    393318 non-null  object        
 6   provider_zip                      393318 non-null  object        
 7   provider_phone_number             393318 non-null  object        
 8   county                            393318 non-null  object        
 9   submitted_data                    393318 non-null  object        
 10  data_passed_qa                 

In [64]:
nh_latest_sub_end_22.sort_values(by = ['federal_provider_number','week_ending']).head()


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start
896818,2022-07-10,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,51.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,51.0,48.0,2.0,NaN,94.118,44.0,NaN,81.0,71.0,9.0,NaN,87.654,21.0,NaN,25.926,29.577,2022-07-04
911966,2022-07-17,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,53.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,53.0,51.0,1.0,NaN,96.226,33.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-07-11
927110,2022-07-24,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,54.0,52.0,1.0,NaN,96.296,47.0,NaN,80.0,70.0,9.0,NaN,87.500,34.0,NaN,42.500,48.571,2022-07-18
942253,2022-07-31,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,53.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,54.0,52.0,1.0,NaN,96.296,35.0,NaN,81.0,70.0,10.0,NaN,86.420,21.0,NaN,25.926,30.000,2022-07-25
957396,2022-08-07,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,1.0,0.0,57.0,52.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,53.0,51.0,1.0,NaN,96.226,35.0,NaN,81.0,70.0,11.0,NaN,86.420,26.0,NaN,32.099,37.143,2022-08-01


In [65]:
nh_latest_sub_end_22['month_year'] = pd.to_datetime(nh_latest_sub_end_22['week_start']).dt.to_period('M')
nh_latest_sub_end_22.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,N,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-12-19,2022-12
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,81.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,82.0,75.0,6.0,NaN,91.463,59.0,NaN,185.0,149.0,31.0,NaN,80.541,1.0,NaN,0.541,0.671,2022-12-19,2022-12
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,35.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,35.0,NaN,86.0,81.0,5.0,NaN,94.186,15.0,NaN,17.442,18.519,2022-12-19,2022-12
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,1.0,0.0,90.0,75.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,75.0,73.0,2.0,NaN,97.333,72.0,NaN,97.0,93.0,2.0,NaN,95.876,30.0,NaN,30.928,32.258,2022-12-19,2022-12
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,86.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,92.0,85.0,4.0,NaN,92.391,65.0,NaN,139.0,115.0,18.0,NaN,82.734,5.0,NaN,3.597,4.348,2022-12-19,2022-12


In [66]:
nh_latest_sub_end_22['rev_pct_res_utd_c19_vax'] = 100*(nh_latest_sub_end_22.num_res_utd_c19_vax/nh_latest_sub_end_22.num_res_in_facility).round(2)
nh_latest_sub_end_22['rev_pct_hcp_utd_c19_vax'] = 100*(nh_latest_sub_end_22.num_hcp_utd_c19_vax/nh_latest_sub_end_22.num_hcp).round(2)
nh_latest_sub_end_22.head()



,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,N,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-12-19,2022-12,89.0,30.0
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,81.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,82.0,75.0,6.0,NaN,91.463,59.0,NaN,185.0,149.0,31.0,NaN,80.541,1.0,NaN,0.541,0.671,2022-12-19,2022-12,72.0,1.0
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,35.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,35.0,NaN,86.0,81.0,5.0,NaN,94.186,15.0,NaN,17.442,18.519,2022-12-19,2022-12,92.0,17.0
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,1.0,0.0,90.0,75.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,75.0,73.0,2.0,NaN,97.333,72.0,NaN,97.0,93.0,2.0,NaN,95.876,30.0,NaN,30.928,32.258,2022-12-19,2022-12,96.0,31.0
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,86.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,92.0,85.0,4.0,NaN,92.391,65.0,NaN,139.0,115.0,18.0,NaN,82.734,5.0,NaN,3.597,4.348,2022-12-19,2022-12,71.0,4.0


In [67]:
#subset to only records where data_passed_qa = Y
nh_latest_sub_end_22_y=nh_latest_sub_end_22[nh_latest_sub_end_22['data_passed_qa']=='Y']
nh_latest_sub_end_22_y.reset_index(drop=True)
nh_latest_sub_end_22_y.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
0,2022-12-25,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,N,53.0,49.0,4.0,NaN,92.453,47.0,NaN,81.0,70.0,10.0,NaN,86.420,24.0,NaN,29.630,34.286,2022-12-19,2022-12,89.0,30.0
1,2022-12-25,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,81.0,1.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0,Y,82.0,75.0,6.0,NaN,91.463,59.0,NaN,185.0,149.0,31.0,NaN,80.541,1.0,NaN,0.541,0.671,2022-12-19,2022-12,72.0,1.0
2,2022-12-25,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,0.0,0.0,50.0,35.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,38.0,38.0,0.0,NaN,100.000,35.0,NaN,86.0,81.0,5.0,NaN,94.186,15.0,NaN,17.442,18.519,2022-12-19,2022-12,92.0,17.0
3,2022-12-25,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,1.0,0.0,90.0,75.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,75.0,73.0,2.0,NaN,97.333,72.0,NaN,97.0,93.0,2.0,NaN,95.876,30.0,NaN,30.928,32.258,2022-12-19,2022-12,96.0,31.0
4,2022-12-25,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,2.0,0.0,103.0,86.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,92.0,85.0,4.0,NaN,92.391,65.0,NaN,139.0,115.0,18.0,NaN,82.734,5.0,NaN,3.597,4.348,2022-12-19,2022-12,71.0,4.0


In [68]:
nh_latest_sub_end_22_y.provider_name = nh_latest_sub_end_22_y.provider_name.str.replace(',','')
nh_latest_sub_end_22_y.provider_name = nh_latest_sub_end_22_y.provider_name.str.replace('"','')

In [69]:
nh_latest_sub_end_22_y.provider_name = nh_latest_sub_end_22_y.provider_name.astype(str)

In [70]:
nh_latest_sub_end_22_y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385292 entries, 0 to 1214593
Data columns (total 45 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   week_ending                       385292 non-null  datetime64[ns]
 1   federal_provider_number           385292 non-null  object        
 2   provider_name                     385292 non-null  object        
 3   provider_address                  385292 non-null  object        
 4   provider_city                     385292 non-null  object        
 5   provider_state                    385292 non-null  object        
 6   provider_zip                      385292 non-null  object        
 7   provider_phone_number             385292 non-null  object        
 8   county                            385292 non-null  object        
 9   submitted_data                    385292 non-null  object        
 10  data_passed_qa                 

In [71]:
nh_latest_sub_end_22_y.provider_name.dtypes

dtype('O')

In [72]:
nh_latest_sub_end_22_y.data_passed_qa.value_counts()

Y    385292
Name: data_passed_qa, dtype: int64

In [73]:
nh_latest_sub_end_22_y.submitted_data.value_counts()

Y    385292
Name: submitted_data, dtype: int64

In [74]:
nh_latest_sub_end_22_y=nh_latest_sub_end_22_y.sort_values(by='week_start')

In [75]:
nh_latest_sub_end_22_y.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
20120,2023-01-01,185260,LITTLE SISTERS OF THE POOR,15 AUDUBON PLAZA DRIVE,LOUISVILLE,KY,40217,5026362300,Jefferson,Y,Y,0.0,0.0,0.0,50.0,38.0,1.0,0.0,Y,Y,0.0,1.0,0.0,0.0,Y,38.0,35.0,1.0,92.105,92.105,27.0,71.053,74.0,63.0,0.0,85.135,85.135,37.0,50.000,50.000,58.730,2022-12-26,2022-12,71.0,50.0
20121,2023-01-01,185261,WURTLAND NURSING AND REHABILITATION,100 WURTLAND AVENUE,WURTLAND,KY,41144,6068360931,Greenup,Y,Y,1.0,0.0,0.0,124.0,101.0,0.0,0.0,NaN,NaN,9.9,0.0,1.0,0.0,Y,109.0,94.0,15.0,86.239,86.239,82.0,75.229,111.0,109.0,1.0,98.198,98.198,0.0,0.000,0.000,0.000,2022-12-26,2022-12,75.0,0.0
20122,2023-01-01,185262,MADISON HEALTH AND REHABILITATION CENTER,131 MEADOWLARK DRIVE,RICHMOND,KY,40475,8596233564,Madison,Y,Y,0.0,0.0,0.0,92.0,69.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,69.0,53.0,15.0,76.812,76.812,26.0,37.681,71.0,71.0,0.0,100.000,100.000,20.0,28.169,28.169,28.169,2022-12-26,2022-12,38.0,28.0
20111,2023-01-01,185249,SIGNATURE HEALTHCARE AT JACKSON MANOR REHAB & ...,96 HIGHWAY 3444,ANNVILLE,KY,40402,6063645197,Jackson,Y,Y,0.0,0.0,0.0,51.0,45.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0,Y,NaN,NaN,NaN,70.213,NaN,NaN,46.809,NaN,NaN,NaN,93.443,NaN,NaN,18.033,NaN,NaN,2022-12-26,2022-12,NaN,NaN
22181,2023-01-01,255350,SINGING RIVER GULFPORT SKILLED NURSING FACILITY,15200 COMMUNITY RD,GULFPORT,MS,39503,2285757000,Harrison,Y,Y,0.0,0.0,0.0,30.0,8.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,Y,NaN,NaN,NaN,65.385,NaN,NaN,3.846,NaN,NaN,NaN,74.863,NaN,NaN,10.383,NaN,NaN,2022-12-26,2022-12,NaN,NaN


### export nh_latest_sub_end_22_y to csv

In [76]:
nh_latest_sub_end_22_y.to_csv("data/data_post_proc/nh_latest_sub_end_22_y.csv", encoding='utf-8', sep = '|',index=False)

### subset to nelson county, ky

In [68]:
nh_latest_sub_end_22_nels_ky = nh_latest_sub_end_22.loc[(nh_latest_sub_end_22['county'] == 'Nelson') & (nh_latest_sub_end_22['provider_state'] == 'KY')].sort_values(by='week_start')
nh_latest_sub_end_22_nels_ky.style.set_sticky(axis='columns')


,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip,provider_phone_number,county,submitted_data,data_passed_qa,res_wk_confirmed_c19,res_wk_all_deaths,res_wk_c19_deaths,num_all_beds,num_occ_beds,staff_wk_conf_c19,staff_wk_c19_deaths,short_nrsg_staff,short_aides,wk_res_confirmed_c19_per_1K,num_staff_new_pos_c19_test,num_res_new_flu,num_staff_pers_new_flu,can_test_all_res_staff_pers,num_res_in_facility,num_res_comp_c19_vax_anytime,num_res_decln_c19_vax_anytime,rec_pct_res_comp_c19_vax_anytime,pct_res_comp_c19_vax_anytime,num_res_utd_c19_vax,rec_pct_res_utd_c19_vax,num_hcp,num_hcp_comp_c19_vax_anytime,num_hcp_decln_c19_vax_anytime,rec_pct_hcp_comp_c19_vax_anytime,pct_hcp_comp_c19_vax_anytime,num_hcp_utd_c19_vax,rec_pct_hcp_utd_c19_vax,pct_hcp_utd_c19_vax,pct_hcp_comp_c19_vax_and_utd,week_start,month_year,rev_pct_res_utd_c19_vax,rev_pct_hcp_utd_c19_vax
871549,2022-07-10 00:00:00,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,0.000000,0.000000,0.000000,100.000000,81.000000,4.000000,0.000000,nan,nan,0.000000,4.000000,0.000000,0.000000,Y,81.000000,68.000000,11.000000,nan,83.951000,1.000000,nan,65.000000,51.000000,14.000000,nan,78.462000,2.000000,nan,3.077000,3.922000,2022-07-04 00:00:00,2022-07,0.010000,0.030000
871731,2022-07-10 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,Y,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,2022-07-04 00:00:00,2022-07,nan,nan
871688,2022-07-10 00:00:00,185342,SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS,708 BARTLEY AVENUE,BARDSTOWN,KY,40004,5023489260,Nelson,Y,Y,3.000000,0.000000,0.000000,65.000000,53.000000,1.000000,0.000000,nan,nan,56.600000,1.000000,0.000000,0.000000,Y,53.000000,48.000000,5.000000,nan,90.566000,29.000000,nan,176.000000,79.000000,9.000000,nan,44.886000,31.000000,nan,17.614000,39.241000,2022-07-04 00:00:00,2022-07,0.550000,0.180000
886696,2022-07-17 00:00:00,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,0.000000,0.000000,0.000000,100.000000,86.000000,1.000000,0.000000,nan,nan,0.000000,1.000000,0.000000,0.000000,Y,86.000000,71.000000,13.000000,nan,82.558000,1.000000,nan,65.000000,51.000000,13.000000,nan,78.462000,2.000000,nan,3.077000,3.922000,2022-07-11 00:00:00,2022-07,0.010000,0.030000
886878,2022-07-17 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505000,Nelson,Y,N,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,Y,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,2022-07-11 00:00:00,2022-07,nan,nan
886835,2022-07-17 00:00:00,185342,SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS,708 BARTLEY AVENUE,BARDSTOWN,KY,40004,5023489260,Nelson,Y,Y,0.000000,0.000000,0.000000,65.000000,51.000000,4.000000,0.000000,nan,nan,0.000000,4.000000,0.000000,0.000000,Y,51.000000,46.000000,5.000000,nan,90.196000,28.000000,nan,184.000000,82.000000,10.000000,nan,44.565000,33.000000,nan,17.935000,40.244000,2022-07-11 00:00:00,2022-07,0.550000,0.180000
901839,2022-07-24 00:00:00,185149,LANDMARK OF BARDSTOWN REHABILITATION AND NURSING,120 LIFE CARE WAY,BARDSTOWN,KY,40004,5023484220,Nelson,Y,Y,1.000000,1.000000,0.000000,100.000000,85.000000,0.000000,0.000000,nan,nan,11.760000,0.000000,0.000000,0.000000,Y,85.000000,69.000000,14.000000,nan,81.176000,1.000000,nan,63.000000,50.000000,12.000000,nan,79.365000,2.000000,nan,3.175000,4.000000,2022-07-18 00:00:00,2022-07,0.010000,0.030000
902021,2022-07-24 00:00:00,185422,FLAGET MEMORIAL HOSPITAL NF,4305 NEW SHEPHERDSVILLE ROAD,BARDSTOWN,KY,40004,5023505

In [69]:
nh_latest_sub_end_22_nels_ky.provider_name.value_counts()

LANDMARK OF BARDSTOWN REHABILITATION AND NURSING     24
FLAGET MEMORIAL HOSPITAL NF                          24
SIGNATURE HEALTHCARE AT COLONIAL REHAB & WELLNESS    24
Name: provider_name, dtype: int64

In [70]:
nh_latest_sub_end_22_nels_ky.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 871549 to 35292
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   week_ending                       72 non-null     datetime64[ns]
 1   federal_provider_number           72 non-null     object        
 2   provider_name                     72 non-null     object        
 3   provider_address                  72 non-null     object        
 4   provider_city                     72 non-null     object        
 5   provider_state                    72 non-null     object        
 6   provider_zip                      72 non-null     object        
 7   provider_phone_number             72 non-null     object        
 8   county                            72 non-null     object        
 9   submitted_data                    72 non-null     object        
 10  data_passed_qa                    71 non-nul

### plot time series of staff up to date c19 vax and residents up to date c19 vax

In [75]:
fig = px.line(nh_latest_sub_end_22.loc[(nh_latest_sub_end_22['county'] == 'Nelson')
                                       & (nh_latest_sub_end_22['provider_state'] == 'KY')
                                       & (nh_latest_sub_end_22['data_passed_qa'] == 'Y')]
                 , x="week_start", y=["rev_pct_res_utd_c19_vax","rev_pct_hcp_utd_c19_vax"],
                 #color="rev_pct_res_utd_c19_vax",
                  markers=True,
                 facet_col="provider_name",facet_col_wrap=1)
#fig.update_yaxes(matches=None)
fig.update_layout(
    autosize=True,
    width=1000,
    height=1000,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()


In [63]:
# df.groupby(['Years','Gender','Country'])[['Salary','Work ex']].mean()
# df.reset_index(inpalce=True) 

nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


14929274.0

In [64]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_res_in_facility.sum()



30696057.0

In [65]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_hcp.sum()


51201927.0

In [66]:
nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='Y'].num_hcp_utd_c19_vax.sum()

16483990.0

### Group to city level

In [67]:
nh_latest_sub_end_22_city_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','county','provider_city','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_city_grp.head()

,week_start,provider_state,county,provider_city,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,ANCHORAGE,Y,237.0,188.0,434.0,201.0
1,2022-07-04,AK,Bethel,BETHEL,Y,14.0,10.0,42.0,35.0
2,2022-07-04,AK,Cordova-Mccarthy,CORDOVA,Y,10.0,6.0,21.0,17.0
3,2022-07-04,AK,Fairbanks,FAIRBANKS,Y,70.0,61.0,154.0,74.0
4,2022-07-04,AK,Juneau,JUNEAU,Y,48.0,29.0,125.0,75.0


In [68]:
nh_latest_sub_end_22_city_grp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183269 entries, 0 to 183268
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   week_start           183269 non-null  datetime64[ns]
 1   provider_state       183269 non-null  object        
 2   county               183269 non-null  object        
 3   provider_city        183269 non-null  object        
 4   data_passed_qa       183269 non-null  object        
 5   num_res_in_facility  183269 non-null  float64       
 6   num_res_utd_c19_vax  183269 non-null  float64       
 7   num_hcp              183269 non-null  float64       
 8   num_hcp_utd_c19_vax  183269 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 12.6+ MB


In [69]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


14929274.0

In [70]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_res_in_facility.sum()


30696057.0

In [71]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_hcp.sum()


51201927.0

In [72]:
nh_latest_sub_end_22_city_grp.loc[nh_latest_sub_end_22_city_grp['data_passed_qa']=='Y'].num_hcp_utd_c19_vax.sum()


16483990.0

In [73]:
# create rate cols
nh_latest_sub_end_22_city_grp['pct_res_utd_c19_vax'] = nh_latest_sub_end_22_city_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_city_grp['num_res_in_facility']
nh_latest_sub_end_22_city_grp['pct_hcp_utd_c19_vax'] = nh_latest_sub_end_22_city_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_city_grp['num_hcp']
nh_latest_sub_end_22_city_grp.head()

,week_start,provider_state,county,provider_city,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,ANCHORAGE,Y,237.0,188.0,434.0,201.0,0.793249,0.463134
1,2022-07-04,AK,Bethel,BETHEL,Y,14.0,10.0,42.0,35.0,0.714286,0.833333
2,2022-07-04,AK,Cordova-Mccarthy,CORDOVA,Y,10.0,6.0,21.0,17.0,0.600000,0.809524
3,2022-07-04,AK,Fairbanks,FAIRBANKS,Y,70.0,61.0,154.0,74.0,0.871429,0.480519
4,2022-07-04,AK,Juneau,JUNEAU,Y,48.0,29.0,125.0,75.0,0.604167,0.600000


### Group to county level

In [93]:
nh_latest_sub_end_22_county_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','county','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_county_grp.head()

,week_start,provider_state,county,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,Y,237.0,188.0,434.0,201.0
1,2022-07-04,AK,Bethel,Y,14.0,10.0,42.0,35.0
2,2022-07-04,AK,Cordova-Mccarthy,Y,10.0,6.0,21.0,17.0
3,2022-07-04,AK,Fairbanks,Y,70.0,61.0,154.0,74.0
4,2022-07-04,AK,Juneau,Y,48.0,29.0,125.0,75.0


In [94]:
nh_latest_sub_end_22_county_grp.loc[nh_latest_sub_end_22_county_grp['data_passed_qa']=='Y'].num_res_utd_c19_vax.sum()


14929274.0

In [95]:
# create rate cols
nh_latest_sub_end_22_county_grp['pct_res_utd_c19_vax']= (nh_latest_sub_end_22_county_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_res_in_facility']).round(2)
nh_latest_sub_end_22_county_grp['pct_hcp_utd_c19_vax'] = (nh_latest_sub_end_22_county_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_county_grp['num_hcp']).round(2)
nh_latest_sub_end_22_county_grp.head()


,week_start,provider_state,county,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,2022-07-04,AK,Anchorage,Y,237.0,188.0,434.0,201.0,0.79,0.46
1,2022-07-04,AK,Bethel,Y,14.0,10.0,42.0,35.0,0.71,0.83
2,2022-07-04,AK,Cordova-Mccarthy,Y,10.0,6.0,21.0,17.0,0.60,0.81
3,2022-07-04,AK,Fairbanks,Y,70.0,61.0,154.0,74.0,0.87,0.48
4,2022-07-04,AK,Juneau,Y,48.0,29.0,125.0,75.0,0.60,0.60


In [96]:
nh_latest_sub_end_22_county_grp_y=nh_latest_sub_end_22_county_grp[nh_latest_sub_end_22_county_grp.data_passed_qa == 'Y'].sort_values(by='week_start').reset_index()
nh_latest_sub_end_22_county_grp_y.head()



,index,week_start,provider_state,county,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,0,2022-07-04,AK,Anchorage,Y,237.0,188.0,434.0,201.0,0.79,0.46
1,2022,2022-07-04,OH,Harrison,Y,188.0,9.0,420.0,51.0,0.05,0.12
2,2023,2022-07-04,OH,Henry,Y,172.0,124.0,336.0,93.0,0.72,0.28
3,2024,2022-07-04,OH,Highland,Y,297.0,72.0,436.0,144.0,0.24,0.33
4,2025,2022-07-04,OH,Hocking,Y,98.0,1.0,129.0,2.0,0.01,0.02


### send nh_latest_sub_end_22_county_grp_y to csv

In [97]:
nh_latest_sub_end_22_county_grp_y.to_csv("data/data_post_proc/nh_latest_sub_end_22_county_grp_y.csv", index=False)

In [99]:
fig = px.line(nh_latest_sub_end_22_county_grp_y.loc[(nh_latest_sub_end_22_county_grp_y['provider_state'] == 'KY') &
                                                  (nh_latest_sub_end_22_county_grp_y['county'] == 'Nelson')],
                 x="week_start", y=["pct_res_utd_c19_vax","pct_hcp_utd_c19_vax"],
                  markers=True,
                 #facet_col="county", facet_col_wrap=7,
                  title="Nursing home residents and staff Covid-19 vaccination rates: Nelson County KY"
             )
#fig.update_yaxes(matches=None)
fig.update_layout(
    #autosize=True,
    #width=1000,
    #height=600,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()

### Group to state level

In [77]:
nh_latest_sub_end_22_state_grp=nh_latest_sub_end_22.groupby(['week_start','provider_state','data_passed_qa'],dropna=False)['num_res_in_facility','num_res_utd_c19_vax','num_hcp','num_hcp_utd_c19_vax'].sum().reset_index()
nh_latest_sub_end_22_state_grp.head()


,week_start,provider_state,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax
0,2022-07-04,AK,Y,720.0,441.0,2132.0,966.0
1,2022-07-04,AL,Y,21048.0,9861.0,33118.0,8309.0
2,2022-07-04,AR,Y,15640.0,8662.0,30756.0,14313.0
3,2022-07-04,AZ,Y,11047.0,3613.0,20802.0,6149.0
4,2022-07-04,CA,Y,92350.0,52985.0,146910.0,111892.0


In [78]:
# create rate cols
nh_latest_sub_end_22_state_grp['pct_res_utd_c19_vax']= (nh_latest_sub_end_22_state_grp['num_res_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_res_in_facility']).round(2)
nh_latest_sub_end_22_state_grp['pct_hcp_utd_c19_vax'] = (nh_latest_sub_end_22_state_grp['num_hcp_utd_c19_vax']/nh_latest_sub_end_22_state_grp['num_hcp']).round(2)
nh_latest_sub_end_22_state_grp.head()


,week_start,provider_state,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,2022-07-04,AK,Y,720.0,441.0,2132.0,966.0,0.61,0.45
1,2022-07-04,AL,Y,21048.0,9861.0,33118.0,8309.0,0.47,0.25
2,2022-07-04,AR,Y,15640.0,8662.0,30756.0,14313.0,0.55,0.47
3,2022-07-04,AZ,Y,11047.0,3613.0,20802.0,6149.0,0.33,0.30
4,2022-07-04,CA,Y,92350.0,52985.0,146910.0,111892.0,0.57,0.76


In [100]:
nh_latest_sub_end_22_state_grp_y=nh_latest_sub_end_22_state_grp[nh_latest_sub_end_22_state_grp.data_passed_qa == 'Y'].sort_values(by='week_start').reset_index()
nh_latest_sub_end_22_state_grp_y.head()


,level_0,index,week_start,provider_state,data_passed_qa,num_res_in_facility,num_res_utd_c19_vax,num_hcp,num_hcp_utd_c19_vax,pct_res_utd_c19_vax,pct_hcp_utd_c19_vax
0,0,0,2022-07-04,AK,Y,720.0,441.0,2132.0,966.0,0.61,0.45
1,29,2,2022-07-04,AR,Y,15640.0,8662.0,30756.0,14313.0,0.55,0.47
2,30,3,2022-07-04,AZ,Y,11047.0,3613.0,20802.0,6149.0,0.33,0.30
3,31,4,2022-07-04,CA,Y,92350.0,52985.0,146910.0,111892.0,0.57,0.76
4,32,5,2022-07-04,CO,Y,14554.0,9485.0,25746.0,13309.0,0.65,0.52


###  send nh_latest_sub_end_22_state_grp_y to csv

In [101]:
nh_latest_sub_end_22_state_grp_y.to_csv("data/data_post_proc/nh_latest_sub_end_22_state_grp_y.csv", index=False)

In [102]:
fig = px.line(nh_latest_sub_end_22_state_grp_y.loc[(nh_latest_sub_end_22_state_grp_y['provider_state'] == 'KY')],
                 x="week_start", y=["pct_res_utd_c19_vax","pct_hcp_utd_c19_vax"],
                  markers=True,
                 #facet_col="county", facet_col_wrap=7,
                  title="Nursing home residents and staff Covid-19 vaccination rates: KY"
             )
#fig.update_yaxes(matches=None)
fig.update_layout(
    #autosize=True,
    #width=1000,
    #height=600,   
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')



for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
        
fig.show()

### What pct of entire dataset passed qa?

In [103]:
#Entire dataset?
nh_latest_sub_end_22['data_passed_qa'].value_counts(normalize=True)

Y    0.99175
N    0.00825
Name: data_passed_qa, dtype: float64

In [ ]:
#By State?
state_qa=nh_latest_sub_end_22.groupby('provider_state')['data_passed_qa'].value_counts(normalize=True).to_frame(name='pct').reset_index()

state_qa.head()

In [ ]:
#Sorting states by qa failed rate descending
state_qa.loc[state_qa['data_passed_qa']=='N'].sort_values(by=['pct'], ascending=False)


In [ ]:
#By State and County?
state_county_qa=nh_latest_sub_end_22.groupby(['provider_state','county'])['data_passed_qa'].value_counts(normalize=True).to_frame(name='pct').reset_index()
state_county_qa.head()


In [ ]:
#Sorting counties by qa failed rate, state descending
state_county_qa.loc[state_county_qa['data_passed_qa']=='N'].sort_values(by=['pct','provider_state'], ascending=[False,True])

In [ ]:
#Sorting counties by state, qa failed rate descending
state_county_qa.loc[state_county_qa['data_passed_qa']=='N'].sort_values(by=(['provider_state','pct']), ascending=[True,False])
                                                                                                                  

In [ ]:
nh_dq_fail=nh_latest_sub_end_22.loc[nh_latest_sub_end_22['data_passed_qa']=='N']
nh_dq_fail.info()

In [ ]:
nh_dq_fail.provider_name.value_counts().sort_index()

In [ ]:
nh_dq_pass=nh_sub.loc[nh_sub['data_passed_qa']=='Y']
nh_dq_pass.head()

In [ ]:
nh_dq_pass.tail()

In [ ]:
nh_dq_pass.info()

In [ ]:
#subset to ky
nh_ky=nh_sub.loc[nh_sub['provider_state']=='KY']
nh_ky.head()

In [ ]:
# who failed dq check and how many for how many occurances (weeks)?
nh_ky.loc[nh_ky['data_passed_qa']=='N'].provider_name.value_counts()

In [ ]:
nh_ky_pass=nh_ky.loc[nh_ky['data_passed_qa']=='Y']
nh_ky_pass.info()

In [ ]:
# subset to jefferson county
jeff_cty_pass=nh_ky_pass.loc[(nh_ky_pass['month_year']>='2021-07') & (nh_ky_pass['data_passed_qa']=='Y') & (nh_ky_pass['county']=='Jefferson')].sort_values(by=['provider_name','week_ending'])

#sort by facility, week ending date, num_res_in_facility
jeff_cty_pass.sort_values(['federal_provider_number','week_ending','num_res_in_facility'])


In [ ]:
jeff_cty_pass.loc[jeff_cty_pass['num_res_in_facility'].isnull()].sort_values('week_ending')

In [ ]:
jeff_cty_pass.federal_provider_number.value_counts()

In [ ]:
# Why does 185301 only have 10 occurrences? 

jeff_cty_pass.loc[jeff_cty_pass['federal_provider_number'] == '185301']

# Regis Woods. Shut down sept 2021 for health and safety violations: 
# https://www.courier-journal.com/story/news/local/2021/09/15/louisville-nursing-home-regis-woods-losing-medicaid-funding-residents-relocated/8349023002/


In [ ]:
# Regis Woods. Shut down sept 2021 for health and safety violations: 
# https://www.courier-journal.com/story/news/local/2021/09/15/louisville-nursing-home-regis-woods-losing-medicaid-funding-residents-relocated/8349023002/


### What did Jefferson county nh staff vax rates and res vax rates look like over the past year? (Median and Mean)


In [ ]:
jeff_cty_pass_avg_med=jeff_cty_pass.groupby(['month_year','county'],dropna=False).agg(
sum_hcp_comp_c19_vax=('num_hcp_comp_c19_vax','sum'),
sum_hcp=('num_hcp','sum'),
sum_res_comp_c19_vax=('num_res_comp_c19_vax','sum'),
sum_res_in_facility=('num_res_in_facility','sum'),
pct_hcp_comp_c19_vax_median=('pct_hcp_comp_c19_vax','median'),
pct_res_comp_c19_vax_median=('pct_res_comp_c19_vax','median')).reset_index()


In [ ]:
jeff_cty_pass_avg_med['pct_hcp_comp_c19_vax_avg'] = jeff_cty_pass_avg_med['sum_hcp_comp_c19_vax']/jeff_cty_pass_avg_med['sum_hcp']*100
jeff_cty_pass_avg_med['pct_res_comp_c19_vax_avg'] = jeff_cty_pass_avg_med['sum_res_comp_c19_vax']/jeff_cty_pass_avg_med['sum_res_in_facility']*100


In [ ]:
jeff_cty_pass_avg_med

### Plot: distribution of staff vax and res vax pcts for Jefferson county by month-yr

In [ ]:
jeff_cty_pass['month_year'] = pd.to_datetime(jeff_cty_pass['month_year'], format='%Y-%m')


fig = px.box(jeff_cty_pass, x='month_year', y='pct_hcp_comp_c19_vax', points='all')

fig.update_layout(
    autosize=False,
    width=1200,
    height=600
)

fig.show()

### WIP see below

In [ ]:
## box plot
jeff_cty_pass_fig = px.box(
    jeff_cty_pass, 
    x = pd.to_datetime('week_start'),
    y = 'pct_hcp_comp_c19_vax',
    title = 'Pct healthcare staff who completed covid-19 vaccination',
    points = 'all', # will display dots next to the boxes 
    labels = {
        'x': pd.to_datetime('week_start'),
        'y': 'pct_hcp_comp_c19_vax'
    }
)

jeff_cty_pass_fig.update_layout(
    plot_bgcolor = 'white',
    xaxis = dict(
        showline = True,
        linecolor = 'black'
    ),
    
    yaxis = dict(
        showline = True,
        showgrid = False,
        linecolor = 'black'
    )
    
)

jeff_cty_pass_fig.show()

## Shapefile data

In [ ]:
# Read in shp file as dataframe
# Nursing home shapefile data source: HIFLD data retrieved from: 
# https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::nursing-homes/about
# Latest update for files: Jun 29, 22

nh_shp_df = gpd.read_file('data/Nursing_Homes_shape/NursingHomes.shp')
nh_shp_df.head()

In [ ]:
nh_shp_df.info()

In [ ]:
nh_shp_df.loc[nh_shp_df['NAME']=='CHEROKEE PARK REHABILITATION']